# Filesystem Stuff

## Paths

Julia provides a lot of built-ins for working with paths in a cross-platformy way.

In [1]:
currdir = pwd()
@show basename(currdir)
@show dirname(currdir)
readme = joinpath(currdir, "README.rst")

basename(currdir) = "administrative-scripting-with-julia"
dirname(currdir) = "/home/ninjaaron/doc"


"/home/ninjaaron/doc/administrative-scripting-with-julia/README.rst"

In [2]:
@show splitext("README.rst")
@show isdir(readme)
@show isfile(readme)
stat(readme)

splitext("README.rst") = ("README", ".rst")
isdir(readme) = false
isfile(readme) = true


StatStruct(mode=0o100644, size=7528)